# Introduction to napari
`napari` is a fast, interactive, multi-dimensional image viewer for Python. It’s designed for browsing, annotating, and analyzing large multi-dimensional images. It’s built on top of Qt (for the GUI), vispy (for performant GPU-based rendering), and the scientific Python stack (numpy, scipy).

`napari` includes critical viewer features out-of-the-box, such as support for large multi-dimensional data, layering, and annotation. By integrating closely with the scientific Python ecosystem, napari can be easily coupled to leading machine learning and image analysis tools (e.g. scikit-image, scikit-learn, PyTorch), enabling more user-friendly automated analysis.

`napari` also supports bidirectional communication between the viewer and the Python kernel, which is especially useful when launching from jupyter notebooks or when using our built-in console. Using the console allows you to interactively load and save data from the viewer and control all the features of the viewer programmatically.

This first part is all about getting to know the core functionality of napari. You can launch napari from the console by activating your python environment (e.g. `conda activate napari-tutorial`) and then launching napari, just type `napari`. Or you can launch napari from within a jupyter notebook (great if you want to interact with napari from python and follow along in the tutorial).

In [ ]:
import napari
import imageio
from pathlib import Path

In [ ]:
viewer = napari.Viewer()

### Image loading
There are a few different ways to load images to into our `viewer`.

- By `dragging and dropping` image files onto the viewer
- By selection image files from the `Open File(s)` menu option
- Using the `viewer.open` command with a file path from within the notebook
- Loading the image data into an array and then passing that array using the `viewer.add_image` command

For the first three options the file path will get passed through our fileIO plugin interface, allowing you to easily leverage highly customized fileIO plugins for your diverse needs. The fourth option allows you complete control over loading and visualization and is most suited for when you have data already loaded into your notebook from other sources.

Here we will explore the fourth option, explicitly loading an overview image from the lab and the `add_image()` method of our `Viewer` object.

This works with many different image types of course.
One small caveat about overview images: They can be quite large.
Maximal size of images that napari can directly load without downsampling:  
16384 in any directio max. So if your overview is 21599 x 25599, you can't see the full res data by just dragging & dropping images into napari (=> pyramidial file formats & lazy loading, see OME-Zarr)

In [ ]:
image_folder = Path('/path/to/folder')
img_filename = 'ImageName.tif'

# For example:
image_folder = Path('/Users/joel/Desktop/TIF_OVR_MIP/')
img_filename = '220323SB050AAB_220326_215615_B02_T0001F001L01A01Z01C01.tif'
img = imageio.v2.imread(image_folder / img_filename)

In [ ]:
viewer.add_image(img)

### Changing viewer settings
That's our image. But it doesn't look very nice just now. Let's rescale it better by changing the `contrast limits` in the viewer to something reasonable. And let's change the colormap from grey to blue in the colormap dropdown menu.

We can do this interactively, but we can also do this directly when adding an image. Let's add a second image, but already rescale it and assign a different colormap.

In [ ]:
img_filename_2 = '220323SB050AAB_220326_215615_B02_T0001F001L01A02Z01C02.tif'
img_2 = imageio.v2.imread(image_folder / img_filename_2)

In [ ]:
viewer.add_image(img_2, contrast_limits = [115, 1000], colormap='green')

### Handling multiple layers
Where did the first image go?
When having multi-color images, we need to decide how they are blended together. What we typically want here is `additive` blending, i.e. to see the combination of the channels. Let's change this by choosing additive in the `blending` dropdown.

And for the next channel we add, we can set this from the beginning as well. The add_image function allows us to set many such parameters. Use the help function to get the details:

In [ ]:
help(viewer.add_image)

Another useful parameter is for example the scale parameter, setting the scale of the image correctly in x & y (especially useful if you want to look at something in 3D and the Z resolution is different than the xy resolution). We could set the scale here as `scale=(0.325, 0.325)` for a 20x Yokogawa image (but we need to do this for all layers, see later).

In [ ]:
img_filename_3 = '220323SB050AAB_220326_215615_B02_T0001F001L01A03Z01C03.tif'
img_3 = imageio.v2.imread(image_folder / img_filename_3)
viewer.add_image(img_3, contrast_limits = [115, 3000], colormap='red', blending='additive')

### Adding label images
`napari` supports seven different layer types, **Image**, **Labels**, **Points**, **Vectors**, **Shapes**, **Surface** and **Tracks**. Each layer corresponds to a different data type and has it's own set of visualizations and interactive controls. We provide an [associated tutorial](https://napari.org/tutorials/index.html) for each layer type to help you get started! 

You can add multiple layers of different types into the viewer and work with them, adjusting their properties and performing analysis.  
Let's try out the label layers!

In [ ]:
# TODO: Load label img
labels_path = Path('/Users/joel/Desktop/TIF_OVR_MIP_SEG/object_v0.4')
labels_name = '220323SB050AAB_220326_215615_B02_T0001F001L01A01Z01C01.tif'
labels = imageio.v2.imread(labels_path / labels_name)
viewer.add_labels(labels)

### Modify the layers from python
You can modify the settings for napari layers both from the interface, as well as directly in python. For example, you can get a list of the layers that are currently in the viewer:

In [ ]:
print(viewer.layers)

In [ ]:
# Let's set a correct scale for all 4 layers:
for layer in viewer.layers:
    layer.scale = (0.325, 0.325)

Once the scale is set correctly, we can add a scalebar. Either in the interface under View; Scalebar; Visible to on, or via python:

In [ ]:
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'

# Conclusions
We have now seen the basic functionality of the napari viewer. Take a few minutes to play around with it, add some of your own images and ask if questions remain. Next, we will check how to save images from the napari viewer and how to record movies of animations